In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.models import Model

import src.utils as nc_utils
import src.metrics as nc_metrics

2024-02-08 16:54:54.996190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 16:54:54.996232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 16:54:54.997604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 16:54:55.006703: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 16:54:56.191008: W tensorflow/compiler/tf2

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [ ]:
def dice_coef(y_true, y_pred, smooth=1e-4):
    y_true = layers.Flatten()(y_true)
    y_pred = layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [3]:
strategy = nc_utils.start_session()

2024-02-08 16:54:58.166471: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 16:54:58.214745: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-08 16:54:58.215011: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

ROOT_DIR = Path(".")
IMAGE_SIZE = 256
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
BATCH_SIZE = 16
LR = 1e-4
EPOCHS = 300

In [5]:
(x_train_paths, y_train_paths), (x_valid_paths, y_valid_paths), (x_test_paths,
 y_test_paths) = nc_utils.load_data(
    data_root_directory=ROOT_DIR,
    split=0.1
)

In [7]:
train_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_train_paths, y_train_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [8]:
validation_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_valid_paths, y_valid_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [9]:
test_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_test_paths, y_test_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# Model creation

In [10]:
# Initiating mode on GPU
with strategy.scope():
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")

    encoder = VGG19(input_tensor=inputs, weights="imagenet", include_top=False)
    # skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    skip_connection_names = ["input_image", "block1_pool", "block2_pool", "block3_pool", "block4_pool"]
    encoder_output = encoder.get_layer("block5_pool").output
    #skip_connection_names = ["input_image", "re_lu", "re_lu_3", "re_lu_7"]
    #encoder_output = encoder.get_layer("re_lu_16").output

    f = [16, 32, 48, 64, 80]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, x_skip])

        x = layers.Conv2D(f[-i], (3, 3), padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = layers.Conv2D(f[-i], (3, 3), padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

    x = layers.Conv2D(1, (1, 1), padding="same")(x)
    x = layers.Activation("sigmoid")(x)

    model = Model(inputs, x)

In [11]:
model.compile(optimizer=tf.keras.optimizers.Nadam(LR),
              loss=BinaryCrossentropy,
              metrics=[nc_metrics.cdc, dice_coef, MeanIoU])

In [12]:
# Defining an Early Stopping and Model Checkpoints
early_stopping = EarlyStopping(monitor = 'val_loss',
                              patience = 5, mode = 'min',
                              restore_best_weights = True)

checkpoint = ModelCheckpoint('unet_best_model.h5',
                            monitor = 'val_loss',
                            save_best_only = True)

In [13]:
train_steps = len(x_train_paths) // BATCH_SIZE
valid_steps = len(x_valid_paths) // BATCH_SIZE

if len(x_train_paths) % BATCH_SIZE != 0:
    train_steps += 1
if len(x_valid_paths) % BATCH_SIZE != 0:
    valid_steps += 1

In [14]:
# Training and Testing Model
try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=[early_stopping, checkpoint])
except Exception as e:
    print("An error occurred:", e)

Epoch 1/30
An error occurred: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: Image not found at path: b'images/tumor (159).png'
Traceback (most recent call last):

  File "/home/leonardo/python_venvs/NeuroCovid-Classification-3-10/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/leonardo/python_venvs/NeuroCovid-Classification-3-10/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/tmp/__autograph_generated_fileiy6jxwlm.py", line 37, in _read_and_process_image_and_mask
    image_array = ag__.converted_call(ag__.ld(read_image), (), dict(image_path=ag__.ld(sub_image_path), image_shape=(ag__.ld(image_size), ag__.ld(image_size)), grayscale=False), fscope_1)

  File "/home/leonardo/python_venvs/NeuroCovid-Classification-3-10/lib/python3.10/site-package

[ WARN:0@13.932] global loadsave.cpp:248 findDecoder imread_('b'images/tumor (159).png''): can't open/read file: check file path/integrity
2024-02-08 16:55:11.672181: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: FileNotFoundError: Image not found at path: b'images/tumor (159).png'
Traceback (most recent call last):

  File "/home/leonardo/python_venvs/NeuroCovid-Classification-3-10/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/leonardo/python_venvs/NeuroCovid-Classification-3-10/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/tmp/__autograph_generated_fileiy6jxwlm.py", line 37, in _read_and_process_image_and_mask
    image_array = ag__.converted_call(ag__.ld(read_image), (), dict(image_path=ag__.ld(sub_image_path), image_shape=(ag__.ld(image_size), ag__.ld(image_size)), grayscale=False), fscope_1)

  File "/hom